In [2]:
# %%capture

# %pip install --upgrade transformers datasets jsonlines ai21 langchain chromadb boto3

In [3]:
import transformers
from datasets import load_dataset

dataset = load_dataset("pasinit/scotus",
                      split='train')

/Users/famestad/.pyenv/versions/3.11.3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# dataset.select([1, 2])[0]

In [5]:
sample_texts = [it['text'] for it in dataset.select([3, 6, 9])]

In [6]:
# sample_texts[0]

In [7]:
import ai21

api_key = 'gAXhJl0FdF9SvRIdd90WYFoXBIk1uilx'

ai21.api_key = api_key

In [8]:
import json

- Who is the plaintiff?
- Who is the defendant?
- What is this case about?
- Why is this case important?
- Summarize the plaintiffs argument
- Summarize the defendants argument
- Extract the list of key points in the plaintiffs argument
- Extract the list of key points in the defendants argument
- Identify uncontested facts (occur in both arguments)
- Identify contested facts (conflicts between the arguments)
- Summary of the judgement
- Summary of dissenting argument

In [26]:
import json

def summarize(text, focus=None):
    response = ai21.Summarize.execute(
        source=text,
        sourceType="TEXT",
        focus=focus
    )
    return response['summary']

summarize(sample_texts[0])

"Petitioner was convicted of robbing the manager, but was acquitted on retrial. He was then found guilty of robbing the customer.\nThe double jeopardy provision vitiates petitioner's conviction. He was retried on the same charge and acquitted, but was given a 30-year sentence.\nThis Court held in Ashe v. Swenson that collateral estoppel is fully applicable to the States and that the identity of the robber was the sole disputed issue at each of his trials.\nThe Supreme Court of Florida declined to review Simpson v. State, and the case was remanded to the district court for further proceedings not inconsistent with this opinion.\nThe robbery of the manager and the robbery of the customer grew out of one criminal episode."

In [27]:
summarize(sample_texts[0],
         focus='decision')

"Petitioner was acquitted of robbing the manager, but was found guilty of robbing the customer. The District Court of Appeal held that the double collateral estoppel provision vitiated his conviction.\nA conviction was reversed on appeal because the trial judge neglected to instruct the jury on the lesser-included offense of larceny.\nIn 1968 petitioner was acquitted on the same charge and subsequently charged with robbing a customer. He appealed to the District Court of Appeal and this Court held that collateral estoppel is fully applicable to the States.\nIn Ashe, the court held that a previous judgment of acquittal based on a general verdict could be reexamined to determine whether a rational jury could have grounded its verdict upon an issue other than that which the defendant seeks to foreclose from consideration.\nPetitioner could not have been retried a third time on the charge of robbing the store manager because he had previously secured a jury verdict of guilty and one of acq

**Petitioner was acquitted** of robbing the manager, **but was found guilty of robbing the customer**. The District Court of Appeal **held that the double collateral estoppel provision vitiated his conviction**.\nA **conviction was reversed** on appeal because the trial judge neglected to instruct the jury on the lesser-included offense of larceny.\nIn 1968 petitioner was acquitted on the same charge and subsequently charged with robbing a customer. He appealed to the District Court of Appeal and this Court held that collateral estoppel is fully applicable to the States.\nIn Ashe, the **court held that a previous judgment of acquittal based on a general verdict could be reexamined** to determine whether a rational jury could have grounded its verdict upon an issue other than that which the defendant seeks to foreclose from consideration.\nPetitioner could not have been retried a third time on the charge of robbing the store manager because he had previously secured a jury verdict of guilty and one of acquittal.\nThe **judgment of the Florida District Court of Appeal is vacated and the case is remanded to that court**.\nI agree with the Court's disposition, but would also hold that the Double Jeopardy Clause prohibited Florida from prosecuting petitioner for the robbery of the customer.

In [28]:
summarize(sample_texts[0],
         focus='facts')

"The factual situation presented in Ashe remarkably parallels that of the instant case. The defendant was acquitted of robbing one of the poker players, but was later tried and convicted of robbing another.\nThe facts of this case support the Court's decision that the Double Jeopardy Clause prohibited Florida from prosecuting petitioner for the robbery of the customer."

In [34]:
summarize(sample_texts[0],
         focus='petitioner argument')

"Petitioner was convicted of robbing the manager, but on retrial he was acquitted. The State Supreme Court denied review, and the case was vacated and remanded.\nIn 1968 petitioner was retried on the same charge and acquitted. He was then charged with robbing a customer and found guilty.\nPetitioner was acquitted on a charge of robbing one of the poker players, but was later tried and convicted of robbing a second.\nPetitioner contends that his identity as one of the robbers was the sole disputed issue at each of his trials, but the District Court of Appeal declined to examine the record of the second trial.\nThe ground upon which the state court resolved petitioner's contention is plainly not tenable.\nPetitioner could not have been retried a third time on the charge of robbing the store manager because he had previously secured a jury verdict of guilty as well as one of acquittal."

Petitioner was convicted of robbing the manager, but on retrial he was acquitted. The State Supreme Court denied review, and the case was vacated and remanded.\nIn 1968 petitioner was retried on the same charge and acquitted. He was then charged with robbing a customer and found guilty.\nPetitioner was acquitted on a charge of robbing one of the poker players, but was later tried and convicted of robbing a second.\n**Petitioner contends that his identity as one of the robbers was the sole disputed issue at each of his trials**, but the District Court of Appeal declined to examine the record of the second trial.\nThe ground upon which the state court resolved petitioner's contention is plainly not tenable.\nPetitioner could not have been retried a third time on the charge of robbing the store manager because he had previously secured a jury verdict of guilty as well as one of acquittal.

In [40]:
def build_focus_summaries(text, foci):
    result = []
    for focus in foci:
        try:
            response = ai21.Summarize.execute(
                source = text,
                sourceType = "TEXT",
                focus = focus
            )
            result.append(response.summary)
        except:
            pass
    return result

foci = ["plaintiff", "respondant arguments", "outcome", "judgement", "dissenting judgement", "footnotes"]
summaries = build_focus_summaries(sample_texts[0], foci)

for idx, summary in enumerate(summaries):
    print(f"""- {foci[idx]}: {summary}\n""")

Calling POST https://api.ai21.com/studio/v1/summarize failed with a non-200 response code: 422


- plaintiff: Petitioner was tried and convicted of robbing the manager, but on retrial he was acquitted. The State Supreme Court denied review, and the case was vacated and remanded.
Petitioner was convicted of armed robbery of a store in Jacksonville, Florida, but the conviction was reversed on appeal.
In 1968, petitioner was retried on the same charge and acquitted. He was then charged with robbing a customer and found guilty.
The defendant in Ashe argued that the prohibition against double jeopardy operated as a bar to the second prosecution because the only issue in each trial was the identity of the robbers.
Petitioner contends that his identity as one of the robbers was the sole disputed issue at each of his trials, but the District Court of Appeal declined to examine the record of the second trial.
The state court resolved petitioner's contention on the ground of mutuality.
Petitioner could not have been retried a third time on the charge of robbing the store manager because he 

In [41]:
# Using Simple Regex Based on Section Breaks in Document

import re

pattern = r"^\s*[A-Z]{2,}\s*$"

matches = re.findall(pattern, sample_texts[0], re.MULTILINE)
for match in matches:
    print("Match:", match.strip())

chunks = re.split(pattern, sample_texts[0], flags=re.MULTILINE)
filtered_chunks = [s for s in chunks if len(s) > 0]
for chunk in filtered_chunks:
    print(len(chunk))

6873
